In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import kagglehub
path = kagglehub.dataset_download("hijest/englishrussian-dictionary-for-machine-translate")
print("Path to dataset files:", path)

Using Colab cache for faster access to the 'englishrussian-dictionary-for-machine-translate' dataset.
Path to dataset files: /kaggle/input/englishrussian-dictionary-for-machine-translate


In [ ]:
import string
import re
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
import pandas as pd

# Путь к вашему файлу
text = "/kaggle/input/englishrussian-dictionary-for-machine-translate/rus.txt"

# Чтение файла и обработка строк
with open(text, encoding='utf-8') as file:
    lines = file.read().split("\n")

pairs = []

for line in lines:
    # Ищем разделитель табуляции
    parts = line.split("\t")
    if len(parts) >= 2:
        english, russian = parts[0], parts[1]
        # Добавляем специальные токены для начала и конца предложения
        russian = "[start] " + russian + " [end]"
        pairs.append((english, russian))

In [ ]:
df = pd.DataFrame(pairs)
df.head(20)

,0,1
0,Go.,[start] Марш! [end]
1,Go.,[start] Иди. [end]
2,Go.,[start] Идите. [end]
3,Hi.,[start] Здравствуйте. [end]
4,Hi.,[start] Привет! [end]
5,Hi.,[start] Хай. [end]
6,Hi.,[start] Здрасте. [end]
7,Hi.,[start] Здоро́во! [end]
8,Run!,[start] Беги! [end]
9,Run!,[start] Бегите! [end]


In [ ]:
import random
print(random.choice(pairs))
print(len(pairs))

('Keep looking.', '[start] Смотрите дальше. [end]')
363386


In [ ]:
random.shuffle(pairs)
num_val_samples = int(0.2 * len(pairs))
num_train_samples = len(pairs) - 2 * num_val_samples
train_pairs = pairs[:num_train_samples]
val_pairs = pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = pairs[num_train_samples + num_val_samples:]

In [ ]:
print(len(train_pairs))
print(len(val_pairs))
print(len(test_pairs))

218032
72677
72677


In [ ]:
strip_chars = string.punctuation + "." + "," + "?" + "!"

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

In [ ]:
custom_standardization(pairs)

<tf.Tensor: shape=(363386, 2), dtype=string, numpy=
array([[b'theres only one problem',
        b'start \xd0\x95\xd1\x81\xd1\x82\xd1\x8c \xd1\x82\xd0\xbe\xd0\xbb\xd1\x8c\xd0\xba\xd0\xbe \xd0\xbe\xd0\xb4\xd0\xbd\xd0\xb0 \xd0\xbf\xd1\x80\xd0\xbe\xd0\xb1\xd0\xbb\xd0\xb5\xd0\xbc\xd0\xb0 end'],
       [b'tom is hypocritical',
        b'start \xd0\xa2\xd0\xbe\xd0\xbc  \xd0\xbb\xd0\xb8\xd1\x86\xd0\xb5\xd0\xbc\xd0\xb5\xd1\x80 end'],
       [b'i live alone in this house',
        b'start \xd0\xaf \xd0\xb6\xd0\xb8\xd0\xb2\xd1\x83 \xd0\xbe\xd0\xb4\xd0\xb8\xd0\xbd \xd0\xb2 \xd1\x8d\xd1\x82\xd0\xbe\xd0\xbc \xd0\xb4\xd0\xbe\xd0\xbc\xd0\xb5 end'],
       ...,
       [b'tom said mary was nervous',
        b'start \xd0\xa2\xd0\xbe\xd0\xbc \xd1\x81\xd0\xba\xd0\xb0\xd0\xb7\xd0\xb0\xd0\xbb \xd1\x87\xd1\x82\xd0\xbe \xd0\x9c\xd1\x8d\xd1\x80\xd0\xb8 \xd0\xbd\xd0\xb5\xd1\x80\xd0\xb2\xd0\xbd\xd0\xb8\xd1\x87\xd0\xb0\xd0\xb5\xd1\x82 end'],
       [b'will you give me a job',
        b'start \xd0\xa2\xd1\x8b \xd0\